# Работа с ресурсами

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 14 ноября 10:00**

# Менеджер контекста для смены директории (cd)

Напишите класс менеджера контекста ChangeDir, который временно меняет текущую рабочую директорию на заданную. После выхода из контекста рабочая директория должна вернуться к предыдущей.

**Условия:**
1.	При входе в блок with менеджер контекста должен изменить текущую директорию на указанную.
2.	При выходе из блока with менеджер контекста должен вернуть рабочую директорию на исходное значение.
3.	Обработайте ситуацию, когда указанный путь не существует, с выводом сообщения об ошибке.

**Пример:**

```python
import os

print("Начальная директория:", os.getcwd())

with ChangeDir("/path/to/new/directory"):
    print("Внутри менеджера:", os.getcwd())

print("После выхода:", os.getcwd())
```

In [1]:
import os

In [19]:
class ChangeDir:
    def __init__(self, path):
        self.original_path = os.getcwd()
        self.path = path
    
    def __enter__(self):
        try:
            os.chdir(self.path)
        except FileNotFoundError:
            print(f"error: path '{self.path}' doesn't exist")
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        os.chdir(self.original_path)

In [21]:
# example
print("Начальная директория:", os.getcwd())
try:
    with ChangeDir("../practice"):
        print("Внутри менеджера:", os.getcwd())
except FileNotFoundError as e:
    print(e)

print("После выхода:", os.getcwd())

Начальная директория: /Users/tryadovoi/Desktop/itmo/yadro_school/lesson_06/homework
Внутри менеджера: /Users/tryadovoi/Desktop/itmo/yadro_school/lesson_06/practice
После выхода: /Users/tryadovoi/Desktop/itmo/yadro_school/lesson_06/homework


In [24]:
def all_test_1():
    initial_dir = os.getcwd()
    
    # test 1
    existing_dir = initial_dir 
    with ChangeDir(existing_dir):
        result = os.getcwd()
        assert result == existing_dir, f"Expected {existing_dir}, but got {result}"
    assert os.getcwd() == initial_dir, f"Expected {initial_dir}, but got {os.getcwd()}"

    # test 2
    not_existing_dir = "/path/that/does/not/exist"
    with ChangeDir(not_existing_dir):
        result = os.getcwd()
        assert result == initial_dir, f"Expected {initial_dir} when entering non-existent directory, but got {result}"
    assert os.getcwd() == initial_dir, f"Expected {initial_dir}, but got {os.getcwd()}"

    # test 3
    nested_dir = existing_dir
    with ChangeDir(nested_dir):
        assert os.getcwd() == nested_dir, f"Expected {nested_dir}, but got {os.getcwd()}"
        with ChangeDir(initial_dir):
            assert os.getcwd() == initial_dir, f"Expected {initial_dir} in nested context, but got {os.getcwd()}"
        assert os.getcwd() == nested_dir, f"Expected {nested_dir} after nested context, but got {os.getcwd()}"
    assert os.getcwd() == initial_dir, f"Expected {initial_dir} after all contexts, but got {os.getcwd()}"

    return "everything is working fine!"

print(all_test_1())


error: path '/path/that/does/not/exist' doesn't exist
everything is working fine!


# Перенаправления вывода в файл

Напишите класс менеджера контекста RedirectOutput, который временно перенаправляет стандартный поток вывода stdout в указанный файл. После выхода из контекста вывод должен возвращаться в стандартный поток.

**Условия:**

1.	При входе в блок with менеджер контекста должен перенаправить вывод print в файл, указанный при создании объекта.
2.	При выходе из блока with вывод должен возвращаться в стандартный поток.
3.	Если файл уже существует, вывод должен дописываться к нему, а не перезаписывать его.

**Пример:**
```python
print("Это стандартный вывод")  # Должно выводиться в консоль

with RedirectOutput("output.txt"):
    print("Это вывод в файл")   # Должно записываться в файл "output.txt"

print("Снова стандартный вывод")  # Должно выводиться в консоль
```


In [26]:
import sys

In [27]:
class RedirectOutput:
    def __init__(self, filename):
        self.filename = filename
        self.original_output = sys.stdout


    def __enter__(self):
        self.file = open(self.filename, 'a')
        sys.stdout = self.file
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        sys.stdout = self.original_output
        self.file.close()

In [28]:
# example
print("Это стандартный вывод")  # Должно выводиться в консоль

with RedirectOutput("output.txt"):
    print("Это вывод в файл")   # Должно записываться в файл "output.txt"

print("Снова стандартный вывод")  # Должно выводиться в консоль

Это стандартный вывод
Снова стандартный вывод


In [33]:
def all_test_2():
    file1 = 'test_output_1.txt'
    file2 = 'test_output_2.txt'

    for file in [file1, file2]:
        if os.path.exists(file):
            os.remove(file)
    
    # test 1
    with RedirectOutput(file1):
        print("Hello, World!")
    with open(file1, 'r') as f:
        content = f.read().strip()
    assert content == "Hello, World!", f"Expected 'Hello, World!', but got '{content}'"

    # test 2
    initial_stdout = sys.stdout
    with RedirectOutput(file1):
        pass
    assert sys.stdout == initial_stdout, "Expected stdout to be restored to initial state after context manager"

    # test 3
    with RedirectOutput(file1):
        print("output 1")
        with RedirectOutput(file2):
            print("output 2")
        print("Back to first file")

    with open(file1, 'r') as f:
        content_file1 = f.read().strip().splitlines()
    with open(file2, 'r') as f:
        content_file2 = f.read().strip().splitlines()

    assert content_file1 == ["Hello, World!", "output 1", "Back to first file"], f"Unexpected content in {file1}: {content_file1}"
    assert content_file2 == ["output 2"], f"Unexpected content in {file2}: {content_file2}"

    for file in [file1, file2]:
        if os.path.exists(file):
            os.remove(file)

    return "everything is working fine!"

print(all_test_2())

everything is working fine!


# Замер времени выполнения кода

Напишите класс менеджера контекста Timer, который замеряет время выполнения кода внутри блока with. Менеджер должен выводить время выполнения в консоль по завершении блока. Для замера времени используйте модуль time.

**Условия:**
1. При входе в блок with менеджер контекста должен начинать отсчёт времени.
2. При выходе из блока with менеджер должен выводить в консоль время выполнения кода внутри блока в формате "Время выполнения: X.XXX секунд".
3. Опционально: добавить возможность передавать имя таймера при инициализации, чтобы можно было различать результаты замеров, если их несколько.

**Пример:**
```python
import time

with Timer("Задача 1"):
    time.sleep(1)  # Симуляция работы кода
[Задача 1] Время выполнения: 1.001 секунд
    
with Timer("Задача 2"):
    for i in range(1000000):
        pass
[Задача 2] Время выполнения: 0.034 секунд
```

In [35]:
import time

In [36]:
class Timer:
    def __init__(self, task):
        self.task = task

    def __enter__(self):
        self.start = time.perf_counter()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print(f"[{self.task}] Time: {time.perf_counter() - self.start:.3f}")

In [37]:
# example
with Timer("Задача 1"):
    time.sleep(1)

with Timer("Задача 2"):
    for i in range(1000000):
        pass

[Задача 1] Time: 1.001
[Задача 2] Time: 0.043


In [45]:
def all_test_3():
    # test 1
    start = time.perf_counter()
    with Timer("short task"):
        time.sleep(0.1) 
    duration = time.perf_counter() - start
    assert 0.08 <= duration <= 0.16, f"Expected duration around 0.1s, but got {duration:.3f}s"

    # test 2
    start = time.perf_counter()
    with Timer("long task"):
        time.sleep(0.5)
    duration = time.perf_counter() - start
    assert 0.48 <= duration <= 0.53, f"Expected duration around 0.5s, but got {duration:.3f}s"

    # test 3
    start = time.perf_counter()
    with Timer("very short task"):
        pass 
    duration = time.perf_counter() - start
    assert duration < 0.01, f"Expected duration to be very short, but got {duration:.3f}s"

    return "everything is working fine!"

print(all_test_3())

[short task] Time: 0.105
[long task] Time: 0.505
[very short task] Time: 0.000
everything is working fine!


# Поглощение исключения

Напишите класс менеджера контекста SuppressExceptions, который подавляет указанные исключения внутри блока with, не прерывая выполнение программы. Если в блоке возникает исключение, которое не входит в список подавляемых, оно должно быть выброшено обычным образом.

**Условия:**
1.	При инициализации менеджера контекста нужно передавать типы исключений, которые будут подавляться.
2.	Если в блоке with возникает исключение из списка подавляемых, оно должно игнорироваться.
3.	Если возникает исключение, не входящее в список, оно должно быть выброшено.
4.	Опционально: после подавления исключения вывести сообщение о том, какое исключение было подавлено.


**Пример:**
```python
with SuppressExceptions(ZeroDivisionError, ValueError):
    print(1 / 0)  # Это исключение будет подавлено

with SuppressExceptions(TypeError):
    print(1 + "2")  # Это исключение будет подавлено

with SuppressExceptions(IndexError):
    print([1, 2, 3][5])  # Это исключение будет подавлено

print("Программа продолжает работать после блока with")
```

In [47]:
class SuppressExceptions:
    def __init__(self, *args):
        self.suppress_exceptions = args

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is not None:
            if issubclass(exc_type, self.suppress_exceptions):
                print(f"Suppressed exception: {exc_type.__name__}")
                return True 
            else:
                return False  
        return False

In [48]:
# example
with SuppressExceptions(ZeroDivisionError, ValueError):
    print(1 / 0)  # Это исключение будет подавлено

with SuppressExceptions(TypeError):
    print(1 + "2")  # Это исключение будет подавлено

with SuppressExceptions(IndexError):
    print([1, 2, 3][5])  # Это исключение будет подавлено

print("Программа продолжает работать после блока with")

Suppressed exception: ZeroDivisionError
Suppressed exception: TypeError
Suppressed exception: IndexError
Программа продолжает работать после блока with


In [54]:
def all_test_4():
    # test 1
    try:
        with SuppressExceptions(ZeroDivisionError):
            1 / 0 
        result = "No exception"
    except ZeroDivisionError:
        result = "Exception raised"
    assert result == "No exception", f"Expected suppression of ZeroDivisionError, but got '{result}'"

    # test 2
    try:
        with SuppressExceptions(ValueError):
            1 / 0  
        result = "No exception"
    except ZeroDivisionError:
        result = "Exception raised"
    assert result == "Exception raised", "Expected ZeroDivisionError to be raised, but it was suppressed"

    # test 3
    try:
        with SuppressExceptions(ValueError, TypeError):
            int("not a number")
        result = "No exception"
    except ValueError:
        result = "Exception raised"
    assert result == "No exception", "Expected suppression of ValueError, but exception was raised"

    # test 4
    try:
        with SuppressExceptions(ValueError, TypeError):
            x = 5 + 5
        result = "No exception"
    except Exception:
        result = "Exception raised"
    assert result == "No exception", "Expected no exception, but an exception was raised"

    return "everything is working fine!"

print(all_test_4())


Suppressed exception: ZeroDivisionError
Suppressed exception: ValueError
everything is working fine!


# Создание временного файла
Напишите класс менеджера контекста TemporaryFile, который создаёт временный файл при входе в контекст и автоматически удаляет его при выходе. Менеджер должен позволять записывать и читать данные из файла в течение его существования в контексте.

**Условия:**
1.	При входе в блок with менеджер должен создавать временный файл и возвращать его объект для записи и чтения.
2.	При выходе из блока with временный файл должен автоматически удаляться.
3.	Имя файла должно быть уникальным и генерироваться автоматически.

**Пример**
```python
with TemporaryFile() as temp_file:
    temp_file.write(b"Временные данные\n")  # Записываем данные
    temp_file.seek(0)  # Возвращаемся в начало файла
    print(temp_file.read())  # Читаем данные из временного файла

print("Файл автоматически удалён")
```

In [55]:
import os
import random

In [56]:
class TemporaryFile:
    def __init__(self):
        pass

    def __enter__(self):
        while True:
            self.filename = f"temporary_file_{random.randint(0, 100)}.txt"
            if not os.path.exists(self.filename):
                break
        self.file = open(self.filename, 'w+b')
        return self.file

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.file.close()
        os.remove(self.filename)

In [ ]:
# example
with TemporaryFile() as temp_file:
    temp_file.write(b"temporary data\n")  # Записываем данные
    temp_file.seek(0)  # Возвращаемся в начало файла
    print(temp_file.read().decode().rstrip("\n"))  # Читаем данные из временного файла

print("Файл автоматически удалён")

temporary data
Файл автоматически удалён


In [60]:
def all_test_4():
    # test 1
    with TemporaryFile() as temp_file:
        temp_file.write(b"temporary data\n") 
        temp_file.seek(0) 
        content = temp_file.read().decode().strip() 
    assert content == "temporary data", f"Expected 'temporary data', but got '{content}'"

    # test 2
    with TemporaryFile() as temp_file:
        filename = temp_file.name 
    file_exists = os.path.exists(filename)
    assert not file_exists, f"Expected file '{filename}' to be deleted, but it still exists"

    # test 3
    with TemporaryFile() as temp_file1, TemporaryFile() as temp_file2:
        filename1 = temp_file1.name
        filename2 = temp_file2.name
    assert filename1 != filename2, "Expected unique filenames for each TemporaryFile instance"
    assert not os.path.exists(filename1) and not os.path.exists(filename2), \
        "Expected both files to be deleted, but one or both still exist"

    return "everything is working fine!"

print(all_test_4())

everything is working fine!
